In [1]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import (
    _fetch_batch,
    fetch_data,
    fetch_spot_data,
)


In [2]:
df = pd.read_csv('ocean.csv')
df.tail(50)

,entry_time,entry_price,exit_time,exit_price,strike,call_put,dir,exit_type,expiry,pnl,index,entry_type,num_indices,index_leverage,pnl_w_cs,qty,roi
10082,2024-08-21 14:15,52.7000,2024-08-21 14:51,100.1300,50500,C,short,sl,2024-08-21,-47.4300,BNF,original,1,7,-50.4866,23,-0.0012
10083,2024-08-21 14:15,97.8000,2024-08-21 14:58,4.8900,50500,P,short,trgt,2024-08-21,92.9100,BNF,original,1,7,90.8562,23,0.0021
10084,2024-08-22 09:16,34.9500,2024-08-22 14:45,1.7475,24850,C,short,trgt,2024-08-22,33.2025,NIFTY,original,1,10,32.4686,67,0.0022
10085,2024-08-22 09:16,70.2000,2024-08-22 15:23,39.1000,24850,P,short,time,2024-08-22,31.1000,NIFTY,original,1,10,28.9140,67,0.0019
10086,2024-08-22 10:15,32.7000,2024-08-22 15:00,1.6350,24800,P,short,trgt,2024-08-22,31.0650,NIFTY,original,1,10,30.3783,67,0.0020
10087,2024-08-22 10:15,34.1500,2024-08-22 15:23,10.9500,24800,C,short,time,2024-08-22,23.2000,NIFTY,original,1,10,22.2980,67,0.0015
10088,2024-08-22 11:15,30.4500,2024-08-22 11:41,48.7200,24800,C,short,sl,2024-08-22,-18.2700,NIFTY,original,1,10,-19.8534,67,-0.0013
10089,2024-08-22 11:15,19.0500,2024-08-22 15:03,0.9525,24800,P,short,trgt,2024-08-22,18.0975,NIFTY,original,1,10,17.6974,67,0.0012
10090,2024-08-22 12:15,12.1000,2024-08-22 14:58,0.6050,24850,C,short,trgt,2024-08-22,11.4950,NIFTY,original,1,10,11.2409,67,0.0008
10091,2024-08-22 12:15,38.6500,2024-08-22 15:23,39.1000,24850,P,short,time,2024-08-22,-0.4500,NIFTY,original,1,10,-2.0050,67,-0.0001


In [14]:
trade_book = []

df['expiry'] = pd.to_datetime(df['expiry'])
df['entry_time'] = pd.to_datetime(df['entry_time'])
df['exit_time'] = pd.to_datetime(df['exit_time'])
for i in range(0, len(df)):
    print(df.iloc[i]['expiry'])
    strike = df['strike'].iloc[i]
    expiry = df['expiry'].iloc[i].date()
    entry_time = df['entry_time'].iloc[i].time()
    exit_time = df['exit_time'].iloc[i].time()
    index = df['index'].iloc[i]
    asset_class = df['call_put'].iloc[i]
    qty=df['qty'].iloc[i]

    spread = 100 if df.iloc[i]['index'] == 'BNF' else 50
    final_strike = int(round(strike * 1.01 / spread) * spread) if asset_class == 'C' else int(round(strike * 0.99 / spread) * spread)

    hedge_df = await fetch_data(
        index=index.lower(),
        start_date=expiry,
        start_time=entry_time,
        end_date=expiry,
        end_time=exit_time,
        expiry=expiry,
        strike=final_strike,
        asset_class=asset_class,
    )
    # print(len(hedge_df))

    if not isinstance(hedge_df, str) and hedge_df is not None:
        hedge_df = hedge_df.select(['datetime', 'o', 'h', 'l', 'c'])
        hedge_df = hedge_df.to_pandas()
        hedge_entry_price = hedge_df.iloc[0]['o']
        hedge_exit_price = hedge_df.iloc[-1]['o']
    else:
        hedge_entry_price = 0
        hedge_exit_price = 0

    slippage = 0.01 * (df.iloc[i]['entry_price'] + df.iloc[i]['exit_price'])

    trade_1 = {
        'entry_time': entry_time,
        'entry_price': df.iloc[i]['entry_price'],
        'exit_time': exit_time,
        'exit_price': df.iloc[i]['exit_price'],
        'strike': strike,
        'call_put': asset_class,
        'dir': df.iloc[i]['dir'],
        'exit_type': df.iloc[i]['exit_type'],
        'expiry': expiry,
        'pnl': (df.iloc[i]['entry_price'] - df.iloc[i]['exit_price']),
        'index': index,
        'pnl_w_cs': (df.iloc[i]['entry_price'] - df.iloc[i]['exit_price']) - slippage,
        'qty': qty,
        'roi': ((df.iloc[i]['entry_price'] - df.iloc[i]['exit_price']) - slippage) * qty / 1000000,
        }

    trade_2 = {
        'entry_time': entry_time,
        'entry_price': hedge_entry_price,
        'exit_time': exit_time,
        'exit_price': hedge_exit_price,
        'strike': final_strike,
        'call_put': asset_class,
        'dir': 'hedge',
        'exit_type': df.iloc[i]['exit_type'],
        'expiry': expiry,
        'pnl': (hedge_exit_price - hedge_entry_price),
        'index': index,
        'pnl_w_cs': (hedge_exit_price - hedge_entry_price),
        'qty': qty,
        'roi': (hedge_exit_price - hedge_entry_price) * qty / 1000000,
        }
    trade_book.append(trade_1)
    trade_book.append(trade_2)
    # break

final_tb = pd.DataFrame(trade_book)

2017-01-05 00:00:00
2017-01-05 00:00:00
2017-01-05 00:00:00
2017-01-05 00:00:00
2017-01-05 00:00:00
2017-01-05 00:00:00
2017-01-05 00:00:00
2017-01-05 00:00:00
2017-01-05 00:00:00
2017-01-05 00:00:00
2017-01-05 00:00:00
2017-01-05 00:00:00
2017-01-12 00:00:00
2017-01-12 00:00:00
2017-01-12 00:00:00
2017-01-12 00:00:00
2017-01-12 00:00:00
2017-01-12 00:00:00
2017-01-12 00:00:00
2017-01-12 00:00:00
2017-01-12 00:00:00
2017-01-12 00:00:00
2017-01-12 00:00:00
2017-01-12 00:00:00
2017-01-19 00:00:00
2017-01-19 00:00:00
2017-01-19 00:00:00
2017-01-19 00:00:00
2017-01-19 00:00:00
2017-01-19 00:00:00
2017-01-19 00:00:00
2017-01-19 00:00:00
2017-01-19 00:00:00
2017-01-19 00:00:00
2017-01-19 00:00:00
2017-01-19 00:00:00
2017-01-25 00:00:00
2017-01-25 00:00:00
2017-01-25 00:00:00
2017-01-25 00:00:00
2017-01-25 00:00:00
2017-01-25 00:00:00
2017-01-25 00:00:00
2017-01-25 00:00:00
2017-01-25 00:00:00
2017-01-25 00:00:00
2017-01-25 00:00:00
2017-01-25 00:00:00
2017-01-25 00:00:00
2017-01-25 00:00:00


In [15]:
df['roi'].sum()

2.0016388874999755

In [16]:
final_tb['roi'].sum()

1.1894493562499997

In [17]:
final_tb.head()

,entry_time,entry_price,exit_time,exit_price,strike,call_put,dir,exit_type,expiry,pnl,index,pnl_w_cs,qty,roi
0,09:16:00,58.0000,09:34:00,81.2000,18000,C,short,sl,2017-01-05,-23.2000,BNF,-24.5920,64,-0.0016
1,09:16:00,7.3500,09:34:00,9.3500,18200,C,hedge,sl,2017-01-05,2.0000,BNF,2.0000,64,0.0001
2,09:16:00,50.0000,13:54:00,2.5000,18000,P,short,trgt,2017-01-05,47.5000,BNF,46.9750,64,0.0030
3,09:16:00,4.9500,13:54:00,0.1000,17800,P,hedge,trgt,2017-01-05,-4.8500,BNF,-4.8500,64,-0.0003
4,10:15:00,64.7500,12:16:00,97.1250,18000,C,short,sl,2017-01-05,-32.3750,BNF,-33.9937,64,-0.0022


In [18]:
def generate_stats(tb_expiry):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            # "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        # variation = f'{signal_ma} , {no_of_candles}, {tf}'
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            # variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    # overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        # overall_variation,
    ]
    return stats_df8

In [19]:
final_tb['expiry'] = pd.to_datetime(final_tb['expiry'])
final_tb['Trade Year'] = final_tb['expiry'].dt.year
final_tb['ROI%'] = final_tb['roi']*100

In [20]:
stats = generate_stats(final_tb)
stats

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,20.1969,1536,40.4297,0.1089,-0.0540,-1.5479,13.0480
2018,13.9173,1536,40.4948,0.1261,-0.0729,-4.5524,3.0571
2019,17.8242,2328,40.0773,0.1004,-0.0556,-2.2549,7.9048
2020,11.7971,2524,41.9968,0.1778,-0.1236,-7.2049,1.6374
2021,12.5391,3000,35.1333,0.0885,-0.0492,-3.2535,3.8540
2022,18.2047,3148,37.2300,0.1233,-0.0689,-5.4491,3.3409
2023,15.5696,3696,34.6320,0.1071,-0.0515,-4.5543,3.4187
2024,8.8960,2496,36.4183,0.1651,-0.0992,-8.1579,1.0905
Overall,118.9449,20264.0000,37.7566,0.1246,-0.0706,-8.1579,14.5804


In [21]:
final_tb.to_csv('ocean_w_1pct_away_hedge.csv')